# Tensors
Basically a numpy array that is specialized for GPUs. 

In [10]:
import torch, torchvision
import numpy as np

## Making Tensors

In [4]:
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [5]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.9314, 0.9112],
        [0.1374, 0.3319]]) 



In [6]:
shape = (2, 3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.6584, 0.4244, 0.3243],
        [0.2817, 0.7060, 0.1529]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


## Tensor Attributes

In [7]:
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [9]:
if torch.cuda.is_available():
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

Device tensor is stored on: cuda:0


Tensors are pretty similar to numpy arrays, so it really shouldn't be too big of a deal. 

# Using `TORCH.AUTOGRAD`
This is the automatic differentiation used for backprop in PyTorch

In [11]:
# Load pretrained model
model = torchvision.models.resnet18(pretrained=True)
# Simulate an image with 3 channels, and a height/width of 64
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

/work/08002/tjost/ls6/miniconda3/envs/computerVision/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/work/08002/tjost/ls6/miniconda3/envs/computerVision/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home1/08002/tjost/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [12]:
# Forward pass
prediction = model(data)

In [13]:
# Basic loss
loss = (prediction - labels).sum()
# Backward pass
loss.backward()
print(loss)

In [16]:
# Load optimizer
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
# Initiate gradient descent
optim.step()

### Autograd Details - Differentiation

In [39]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

Create a new tensor `Q` from `a` and `b`
$$
Q = 3a^3 - b^2
$$ 

In [41]:
Q = 3*a**3 - b**2

Find partial derivatives:
$$
\frac{\partial{Q}}{\partial{a}} = 9a^2
\\
\frac{\partial{Q}}{\partial{b}} = -2b
$$

Calling `.backward()` on `Q` will calculate the gradients and store them in `.grad`

From tutorial:
"We need to explicitly pass a gradient argument in Q `.backward()` because it is a vector. `gradient` is a tensor of the same shape as Q, and it represents the gradient of Q w.r.t. itself, i.e.

$$
\frac{\partial{Q}}{\partial{Q}} = 1
$$



In [42]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [46]:
print(a.grad)
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


Makes sense! Eg:
$$
\frac{\partial{Q}}{\partial{a}} = 9a^2 = 9*4^2 = 36
$$

We don't have to use autograd on everything. For example, we could finetune a pretrained network by "freezing" most of the model. Let's load and freeze a model. 

In [47]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

For this, the classifier is the last layer called `model.fc`. We can replace it with a new layer which is unfrozen by default. 

In [48]:
model.fc = nn.Linear(512, 10)

In [49]:
# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)